# 🧪 Step 1: Research & Data Modelling
**PR Branch Name:** pharmacies-data-modelling

This notebook documents the process for Step 1 of the "Banks in Berlin" project:
- **1.1 Data Source Discovery**
- **1.2 Modelling & Planning**
- **1.3 Prepare the /sources Directory**
- **1.4 Review**

Goal:
- Identify and document relevant data sources.
- Select the 27 key parameters for our use case.
- Draft the planned table schema.
- Plan cleaning and transformation steps before database population.


## 1.1 Data Source Discovery

**Topic:** Banks in Berlin

**Main source:**
- **Name:** OpenStreetMap (OSM) via OSMnx library
- **Source and origin:** Public crowdsourced geospatial database
- **Update frequency:** Continuous (dynamic)
- **Data type:** Dynamic (API query using `amenity=pharmacies`)
- **Reason for selection:**  
  - Covers all pharmacies in Berlin  
  - Includes coordinates, names, addresses, and other useful attributes  
  - Open, free, and easy to query programmatically

**Optional additional sources:**
- **Name:** Berlin Open Data Portal (daten.berlin.de)
- **Source and origin:** Official Berlin city government
- **Update frequency:** Varies per dataset
- **Data type:** Static or semi-static (download as CSV/GeoJSON)
- **Possible usage:** Enrich with official administrative boundaries or extra metadata

**Enrichment potential:**
- Neighborhood/district info from Berlin shapefiles (GeoJSON)
- Linking to local amenities for spatial context


In [131]:
# Install Libraries

! pip install osmnx geopandas pandas --quiet

In [132]:
# Import Libraries

import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd

In [133]:
# Fetch banks in Berlin from OSM using the tag "amenity=bank"
# tags filter for only features with 

tags = {"amenity": "pharmacy"}

In [134]:
# Fetch geometries for Berlin
# pharmacy-gdf = GeoDataFrame (DataFrame with geometry)

pharmacy_gdf = ox.features_from_place("Berlin, Germany", tags)


In [135]:
# Display basic info

print(f"Number of pharmacy entries fetched: {len(pharmacy_gdf)}")
pharmacy_gdf.head()

Number of pharmacy entries fetched: 675


geometry addr:city addr:country  \
element id                                                           
node    60775323   POINT (13.4866 52.54074)    Berlin           DE   
        60848447  POINT (13.46965 52.53184)       NaN          NaN   
        60852928  POINT (13.46851 52.52756)    Berlin           DE   
        68437791  POINT (13.45057 52.48939)    Berlin          NaN   
        69226035  POINT (13.39586 52.51056)    Berlin           DE   

                 addr:housenumber addr:postcode             addr:street  \
element id                                                                
node    60775323                3         13055    Reichenberger Straße   
        60848447              NaN           NaN                     NaN   
        60852928               11         10369  Rudolf-Seiffert-Straße   
        68437791               46         12435      Karl-Kunger-Straße   
        69226035               43         10117        Leipziger Straße   

                           addr:suburb   amenity check_date:opening_hours  \
element id                                                                  
node    60775323  Alt-Hohenschönhausen  pharmacy               2024-09-26   
        60848447                   NaN  pharmacy               2025-02-11   
        60852928             Fennpfuhl  pharmacy               2024-10-16   
        68437791           Alt-Treptow  pharmacy               2024-06-04   
        69226035                 Mitte  pharmacy                      NaN   

                 dispensing healthcare                    name  \
element id                                                       
node    60775323        yes   pharmacy  Reichenberger Apotheke   
        60848447        yes   pharmacy       Castello-Apotheke   
        60852928        yes   pharmacy          Rosen Apotheke   
        68437791        yes   pharmacy     Margareten-Apotheke   
        69226035        yes   pharmacy      Leipziger Apotheke   

                                                      opening_hours  \
element id                                                            
node    60775323  Mo-Th 08:00-19:00; Fr 08:00-18:30; Sa 09:00-13:00   
        60848447                  Mo-Fr 08:30-19:00; Sa 08:30-14:00   
        60852928                  Mo-Fr 08:00-19:00; Sa 08:00-12:00   
        68437791                  Mo-Fr 08:30-18:30; Sa 08:30-13:00   
        69226035                  Mo-Fr 08:00-19:00; Sa 08:00-14:00   

                           phone toilets:wheelchair  \
element id                                            
node    60775323  +49 30 9713807                 no   
        60848447             NaN                NaN   
        60852928  +49 30 9759449                NaN   
        68437791  +49 30 5337855                NaN   
        69226035             NaN                NaN   

                                                  website wheelchair  \
element id                                                             
node    60775323        https://reichenbergerapotheke.de/        yes   
        60848447                                      NaN        yes   
        60852928                  https://www.zurrose.de/        yes   
        68437791  http://www.apotheke.borchert-online.de/         no   
        69226035       https://www.leipziger-apotheke.de/        yes   

                  check_date      man_made payment:mastercard payment:visa  \
element id                                                                   
node    60775323         NaN           NaN                NaN          NaN   
        60848447         NaN           NaN                NaN          NaN   
        60852928         NaN           NaN                NaN          NaN   
        68437791  2024-09-06           NaN                NaN          NaN   
        69226035  2022-08-22  surveillance                yes          yes   

                 surveillance contact:website level email  fax contact:phone  \

In [79]:
pharmacy_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 675 entries, ('node', np.int64(60775323)) to ('way', np.int64(1410373889))
Columns: 103 entries, geometry to payment:american_express
dtypes: geometry(1), object(102)
memory usage: 567.1+ KB


In [136]:
# List the columns names of the GeoDataFrame
pharmacy_gdf.columns.to_list()


['geometry',
 'addr:city',
 'addr:country',
 'addr:housenumber',
 'addr:postcode',
 'addr:street',
 'addr:suburb',
 'amenity',
 'check_date:opening_hours',
 'dispensing',
 'healthcare',
 'name',
 'opening_hours',
 'phone',
 'toilets:wheelchair',
 'website',
 'wheelchair',
 'check_date',
 'man_made',
 'payment:mastercard',
 'payment:visa',
 'surveillance',
 'contact:website',
 'level',
 'email',
 'fax',
 'contact:phone',
 'drinking_water:refill',
 'operator',
 'addr:city:fa',
 'drive_through',
 'contact:email',
 'owner',
 'contact:fax',
 'wheelchair:description',
 'start_date',
 'source',
 'health_facility:type',
 'medical_system:western',
 'payment:cash',
 'payment:credit_cards',
 'payment:debit_cards',
 'name_old',
 'building:levels',
 'old_name',
 'wheelchair:source',
 'ref:vatin',
 'access:covid19',
 'delivery:covid19',
 'brand',
 'brand:wikidata',
 'name:en',
 'addr:place',
 'dog',
 'opening_hours:signed',
 'takeaway:covid19',
 'payment:girocard',
 'addr:housename',
 'wheelchair:de

In [137]:
# Explore all columns 
pharmacy_gdf.describe(include="all").T


,count,unique,top,freq
geometry,675,675,POINT (13.4866016 52.5407412),1
addr:city,545,1,Berlin,545
addr:country,425,1,DE,425
addr:housenumber,561,240,1,18
addr:postcode,555,175,12043,9
...,...,...,...,...
building:roof,1,1,flat,1
roof:shape,1,1,flat,1
access,2,1,customers,2
room,3,2,shop,2


In [138]:
# check missing values in each column
missing_count = pharmacy_gdf.isna().sum().sort_values(ascending=False)
# List missing values count when missing values greater than 100
print(missing_count[missing_count > 200])


payment:american_express    674
wheelchair:source           674
width                       674
branch                      674
drinking_water:refill       674
                           ... 
website                     387
phone                       375
check_date:opening_hours    336
addr:country                250
addr:suburb                 231
Length: 92, dtype: int64


In [16]:
# check unique values in 'barand' column
pharmacy_gdf['brand'].value_counts()

brand
easyApotheke        12
Linden Apotheken     1
Name: count, dtype: int64

In [83]:
#expland all columns to see more details
pd.set_option('display.max_columns', None)
print(pharmacy_gdf.head(3))

                                   geometry addr:city addr:country  \
element id                                                           
node    60775323   POINT (13.4866 52.54074)    Berlin           DE   
        60848447  POINT (13.46965 52.53184)       NaN          NaN   
        60852928  POINT (13.46851 52.52756)    Berlin           DE   

                 addr:housenumber addr:postcode             addr:street  \
element id                                                                
node    60775323                3         13055    Reichenberger Straße   
        60848447              NaN           NaN                     NaN   
        60852928               11         10369  Rudolf-Seiffert-Straße   

                           addr:suburb   amenity check_date:opening_hours  \
element id                                                                  
node    60775323  Alt-Hohenschönhausen  pharmacy               2024-09-26   
        60848447                   NaN  ph

## 1.2 Modelling & Planning

### Selected 25 Key Columns
1. osm_id
2. name
3. brand
4. operator
5. street
6. housenumber
7. postcode
8. surburb
9. city
10. country
11. phone
12. email
13. website
14. openinghours
15. payment_visa
16. payment_mastercard
17. payment_girocard
18. dispensing
19. delivery
20. wheelchair
21. building
22. latitude
23. longitude
24. geom_type
25. geom
26. neighbourhood
27. district
28. source

---

### How this connects to existing tables:
- **Coordinates (latitude, longitude, geom):** link to neighbourhood and district polygons.
- **Neighbourhood & district fields:** join with administrative boundaries table.
- **Source field:** ensures traceability.

---

### Planned Schema: `pharmacy_in_berlin`
| Column Name     | Data Type | Description | Example |
|-----------------|-----------|-------------|---------|
| osm_id          | int       | Unique OSM ID | 12345678 |
| name            | text      | Bank name | Deutsche Bank |
| brand           | text      | Brand name if available | Sparkasse |
| operator        | text      | Entity operating the bank | Berliner Volksbank |
| street          | text      | Street name | Friedrichstraße |
| housenumber     | text      | House number | 45 |
| postcode        | text      | Postal code | 10117 |
| city            | text      | City name | Berlin |
| country         | text      | Country code | DE |
| phone           | text      | Contact phone | +49 30 123456 |
| email           | text      | Contact email | info@bank.de |
| website         | text      | Website URL | www.bank.de |
| openinghours   | text      | Opening hours string | Mo-Fr 09:00-17:00 |
| atm             | text      | Presence of ATM | yes |
| wheelchair      | text      | Accessibility info | yes |
| building        | text      | Building type | yes |
| latitude        | float     | Latitude coordinate | 52.5200 |
| longitude       | float     | Longitude coordinate | 13.4050 |
| geom_type       | text      | Geometry type | Point |
| geom            | geometry  | Full geometry | (GeoJSON) |
| neighbourhood   | text      | Local neighbourhood name | Mitte |
| district        | text      | Berlin district | Mitte |
| source          | text      | Data source info | OSM |

---

### Known Data Issues
- Missing contact details for some entries.
- Inconsistent postcode and address formats.
- Neighbourhood and district not always included in raw OSM data.
- Opening hours in non-standard formats.

---

### Transformation Plan
1. Fetch data from OSM with filter `amenity=pharmacy` (Berlin bounding box).
2. Clean column names → snake_case.
3. Normalize formats (phone, postcode, website URLs).
4. Enrich with neighbourhood/district via spatial join.
5. Save cleaned dataset (GeoJSON + CSV).


In [ ]:
# Select 25 Columns & Add Coordinates

In [139]:
# Ensure geometry type is Point for lat/lon extraction

pharmacy_gdf = pharmacy_gdf.to_crs(epsg=4326)


In [140]:
pharmacy_gdf['geometry'] = pharmacy_gdf['geometry'].apply(lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point())
#Extract latitude and longitude
pharmacy_gdf["latitude"] = pharmacy_gdf.geometry.y
pharmacy_gdf["longitude"] = pharmacy_gdf.geometry.x
pharmacy_gdf

geometry addr:city addr:country  \
element id                                                             
node    60775323     POINT (13.4866 52.54074)    Berlin           DE   
        60848447    POINT (13.46965 52.53184)       NaN          NaN   
        60852928    POINT (13.46851 52.52756)    Berlin           DE   
        68437791    POINT (13.45057 52.48939)    Berlin          NaN   
        69226035    POINT (13.39586 52.51056)    Berlin           DE   
...                                       ...       ...          ...   
way     510479858   POINT (13.30532 52.61841)    Berlin          NaN   
        739645093   POINT (13.44714 52.50581)       NaN          NaN   
        1002286243  POINT (13.60546 52.53736)    Berlin           DE   
        1208913688  POINT (13.58979 52.51307)       NaN          NaN   
        1410373889  POINT (13.34369 52.52812)    Berlin          NaN   

                   addr:housenumber addr:postcode             addr:street  \
element id                                                                  
node    60775323                  3         13055    Reichenberger Straße   
        60848447                NaN           NaN                     NaN   
        60852928                 11         10369  Rudolf-Seiffert-Straße   
        68437791                 46         12435      Karl-Kunger-Straße   
        69226035                 43         10117        Leipziger Straße   
...                             ...           ...                     ...   
way     510479858                47         13467            Heinsestraße   
        739645093               NaN           NaN                     NaN   
        1002286243               24         12627        Stendaler Straße   
        1208913688              NaN           NaN                     NaN   
        1410373889               14         10551             Stromstraße   

                             addr:suburb   amenity check_date:opening_hours  \
element id                                                                    
node    60775323    Alt-Hohenschönhausen  pharmacy               2024-09-26   
        60848447                     NaN  pharmacy               2025-02-11   
        60852928               Fennpfuhl  pharmacy               2024-10-16   
        68437791             Alt-Treptow  pharmacy               2024-06-04   
        69226035                   Mitte  pharmacy                      NaN   
...                                  ...       ...                      ...   
way     510479858                    NaN  pharmacy                      NaN   
        739645093                    NaN  pharmacy                      NaN   
        1002286243           Hellersdorf  pharmacy                      NaN   
        1208913688                   NaN  pharmacy                      NaN   
        1410373889                   NaN  pharmacy                      NaN   

                   dispensing healthcare                              name  \
element id                                                                   
node    60775323          yes   pharmacy            Reichenberger Apotheke   
        60848447          yes   pharmacy                 Castello-Apotheke   
        60852928          yes   pharmacy                    Rosen Apotheke   
        68437791          yes   pharmacy               Margareten-Apotheke   
        69226035          yes   pharmacy                Leipziger Apotheke   
...                       ...        ...                               ...   
way     510479858         NaN   pharmacy                   Hirsch-Apotheke   
        739645093         yes   pharmacy                    Arena Apotheke   
        1002286243        yes   pharmacy                Kastanien Apotheke   
        1208913688        yes   pharmacy                  Prinzen Apotheke   
        1410373889        NaN   pharmacy  Apotheke im Schultheiss Quartier   

                                                       

In [142]:
# Select the 25 columns (fill missing with None if not present)

selected_columns = [
    #"osmid",
    "name", "brand", "operator",
    "addr:street", "addr:housenumber", "addr:postcode", "addr:suburb","addr:city", "addr:country",
    "phone", "email", "website", "opening_hours",
    "payment:visa", "payment:mastercard","payment:girocard", "dispensing", "delivery","surveillance","wheelchair", "building",
    "latitude", "longitude", "geometry",
    # placeholders for enrichment
    #"neighbourhood", "district",
    # add source info
    "source"
]

In [143]:
# Rename map for only the columns that need renaming

rename_map = {
    "addr:street": "street",
    "addr:housenumber": "housenumber",
    "addr:postcode": "postcode",
    "addr:suburb": "suburb",
    "addr:city": "city",
    "addr:country": "country",
    "payment:visa": "payment_visa",
    "payment:mastercard": "payment_mastercard",
    "payment:girocard": "payment_girocard",
    "opening_hours": "openinghours",
    "wheelchair": "wheelchair_accessible",
    "building": "building_type"
}

In [144]:
# # Add missing columns if they don’t exist in the data
# for col in selected_columns:
#     if col not in pharmacy_gdf.columns:
#         banks_gdf[col] = None

In [145]:
# Select the columns in the right order
pharmacy_df = pharmacy_gdf[selected_columns]

In [146]:
# Rename the columns
pharmacy_df = pharmacy_df.rename(columns=rename_map)

In [147]:
# Preview the final DataFrame
pharmacy_df.head()

name brand operator  \
element id                                                
node    60775323  Reichenberger Apotheke   NaN      NaN   
        60848447       Castello-Apotheke   NaN      NaN   
        60852928          Rosen Apotheke   NaN      NaN   
        68437791     Margareten-Apotheke   NaN      NaN   
        69226035      Leipziger Apotheke   NaN      NaN   

                                  street housenumber postcode  \
element id                                                      
node    60775323    Reichenberger Straße           3    13055   
        60848447                     NaN         NaN      NaN   
        60852928  Rudolf-Seiffert-Straße          11    10369   
        68437791      Karl-Kunger-Straße          46    12435   
        69226035        Leipziger Straße          43    10117   

                                suburb    city country           phone email  \
element id                                                                     
node    60775323  Alt-Hohenschönhausen  Berlin      DE  +49 30 9713807   NaN   
        60848447                   NaN     NaN     NaN             NaN   NaN   
        60852928             Fennpfuhl  Berlin      DE  +49 30 9759449   NaN   
        68437791           Alt-Treptow  Berlin     NaN  +49 30 5337855   NaN   
        69226035                 Mitte  Berlin      DE             NaN   NaN   

                                                  website  \
element id                                                  
node    60775323        https://reichenbergerapotheke.de/   
        60848447                                      NaN   
        60852928                  https://www.zurrose.de/   
        68437791  http://www.apotheke.borchert-online.de/   
        69226035       https://www.leipziger-apotheke.de/   

                                                       openinghours  \
element id                                                            
node    60775323  Mo-Th 08:00-19:00; Fr 08:00-18:30; Sa 09:00-13:00   
        60848447                  Mo-Fr 08:30-19:00; Sa 08:30-14:00   
        60852928                  Mo-Fr 08:00-19:00; Sa 08:00-12:00   
        68437791                  Mo-Fr 08:30-18:30; Sa 08:30-13:00   
        69226035                  Mo-Fr 08:00-19:00; Sa 08:00-14:00   

                 payment_visa payment_mastercard payment_girocard dispensing  \
element id                                                                     
node    60775323          NaN                NaN              NaN        yes   
        60848447          NaN                NaN              NaN        yes   
        60852928          NaN                NaN              NaN        yes   
        68437791          NaN                NaN              NaN        yes   
        69226035          yes                yes              NaN        yes   

                 delivery surveillance wheelchair_accessible building_type  \
element id                                                                   
node    60775323      NaN          NaN                   yes           NaN   
        60848447      NaN          NaN                   yes           NaN   
        60852928      NaN          NaN                   yes           NaN   
        68437791      NaN          NaN                    no           NaN   
        69226035      NaN      outdoor                   yes           NaN   

                   latitude  longitude                   geometry source  
element id                                                                
node    60775323  52.540741  13.486602   POINT (13.4866 52.54074)    NaN  
        60848447  52.531835  13.469654  POINT (13.46965 52.53184)    NaN  
        60852928  52.527555  13.468513  POINT (13.46851 52.52756)    NaN  
        68437791  52.489390  13.450570  POINT (13.45057 52.48939)    NaN  
        69226035  52.510556  13.395863  POINT (13.39586 52.51056)    NaN

## Step 1 Review and A–F Data Familiarization

### A) Quick overview

In [148]:
# How many rows and columns?
# pharmacy_df.shape

print("Rows, Columns:", pharmacy_df.shape)

Rows, Columns: (675, 25)


In [149]:
# What are the column names (in order)?
# banks_df.columns.tolist()

print("\nColumns:", pharmacy_df.columns.tolist())


Columns: ['name', 'brand', 'operator', 'street', 'housenumber', 'postcode', 'suburb', 'city', 'country', 'phone', 'email', 'website', 'openinghours', 'payment_visa', 'payment_mastercard', 'payment_girocard', 'dispensing', 'delivery', 'surveillance', 'wheelchair_accessible', 'building_type', 'latitude', 'longitude', 'geometry', 'source']


In [150]:
# Data types and non-null counts

pharmacy_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 675 entries, ('node', np.int64(60775323)) to ('way', np.int64(1410373889))
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   name                   673 non-null    object  
 1   brand                  13 non-null     object  
 2   operator               141 non-null    object  
 3   street                 565 non-null    object  
 4   housenumber            561 non-null    object  
 5   postcode               555 non-null    object  
 6   suburb                 444 non-null    object  
 7   city                   545 non-null    object  
 8   country                425 non-null    object  
 9   phone                  300 non-null    object  
 10  email                  91 non-null     object  
 11  website                288 non-null    object  
 12  openinghours           633 non-null    object  
 13  payment_visa           18 non-null

### B) Missing values per column

In [151]:
# Count missing values (NaN/None) in each column
# I need this to compute percentages of missing values below

missing_count = pharmacy_df.isna().sum().sort_values(ascending=False)
print(missing_count)


delivery                 673
building_type            670
payment_girocard         669
surveillance             668
brand                    662
payment_mastercard       657
payment_visa             657
source                   651
email                    584
operator                 534
website                  387
phone                    375
country                  250
suburb                   231
city                     130
postcode                 120
housenumber              114
street                   110
dispensing                90
wheelchair_accessible     57
openinghours              42
name                       2
latitude                   0
longitude                  0
geometry                   0
dtype: int64


In [152]:
# Number of rows (observations, banks)
# I need this to compute percentages of missing values below

row_count = len(pharmacy_df)
print(row_count)


675


In [153]:
# Build table with counts and % of missing values
# What does pd.DataFrame({...}) do? It converts that dictionary into a DataFrame (like an Excel table).
# The keys become column names.
# The values become column data.

missing = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": (missing_count / row_count * 100).round(1)
}).sort_values(by="missing_pct", ascending=False)

print(missing)

                       missing_count  missing_pct
delivery                         673         99.7
building_type                    670         99.3
payment_girocard                 669         99.1
surveillance                     668         99.0
brand                            662         98.1
payment_mastercard               657         97.3
payment_visa                     657         97.3
source                           651         96.4
email                            584         86.5
operator                         534         79.1
website                          387         57.3
phone                            375         55.6
country                          250         37.0
suburb                           231         34.2
city                             130         19.3
postcode                         120         17.8
housenumber                      114         16.9
street                           110         16.3
dispensing                        90         13.3


### C) Distinct values per column

In [98]:
# Number of unique values per column
# Goal: See the “variety” of each column.


distinct = pharmacy_df.nunique().sort_values(ascending=False)
print(distinct)

# Concusion:
# latitude, longitude and geometry are diverse  => Columns to keep but use mainly for mapping
# country, city , email , maybe source  => Columns I might drop/ignore later (in Step2)
# brand, operator, postcode, wheelchair, atm, maybe opening_hours => Columns that will be most useful in Step 2


latitude                 675
geometry                 675
longitude                675
name                     629
street                   369
openinghours             339
phone                    300
website                  286
housenumber              240
postcode                 175
operator                 137
email                     91
suburb                    84
source                    10
dispensing                 3
surveillance               3
wheelchair_accessible      3
building_type              3
brand                      2
payment_mastercard         1
city                       1
delivery                   1
payment_visa               1
country                    1
payment_girocard           1
dtype: int64


### D) Most common values in key columns

In [99]:
# Goal: Peek at distributions, not just counts.

# Example: top 10 brands
print("\nTop 10 brands:")
print(pharmacy_df["brand"].value_counts().head(10))


Top 10 brands:
brand
easyApotheke        12
Linden Apotheken     1
Name: count, dtype: int64


In [154]:
# Example: top 10 operators
print("\nTop 10 operators:")
print(pharmacy_df["operator"].value_counts().head(10))



Top 10 operators:
operator
Lars Rieck e.K.          2
Ralf Goepfert e.K.       2
Witzleben Apotheke       2
Christian Melzer e.K.    2
Herr S. Stoof            1
Stefan Rezepa e.K        1
Ute Nitzer               1
Claudia Spieß e.K.       1
Jürgen Drescher          1
Anna Fredrich e.K.       1
Name: count, dtype: int64


In [155]:
# Example: most common street 
print("\nTop street:")
print(pharmacy_df["street"].value_counts().head(10))


Top street:
street
Karl-Marx-Straße        11
Schloßstraße             9
Müllerstraße             7
Hauptstraße              7
Badstraße                6
Mariendorfer Damm        6
Kurfürstendamm           6
Hermannstraße            6
Wilmersdorfer Straße     6
Bahnhofstraße            6
Name: count, dtype: int64


In [156]:
# Example: most common postcode 
print("\nTop postcode:")
print(pharmacy_df["postcode"].value_counts().head(10))


Top postcode:
postcode
12043    9
10719    8
10117    8
12163    8
12627    7
13353    7
10627    7
13357    7
14199    7
10365    6
Name: count, dtype: int64


In [157]:
# Example: most common opening_hours
print("\nTop opening_hours:")
print(pharmacy_df["openinghours"].value_counts().head(10))


Top opening_hours:
openinghours
Mo-Fr 08:30-18:30; Sa 08:30-13:00    35
Mo-Fr 08:30-19:00; Sa 08:30-14:00    21
Mo-Fr 08:30-18:30; Sa 08:30-13:30    17
Mo-Fr 08:30-18:30; Sa 09:00-14:00    13
Mo-Fr 08:00-19:00; Sa 09:00-14:00    12
Mo-Fr 08:00-18:30; Sa 08:00-13:00    12
Mo-Fr 08:30-18:30; Sa 09:00-13:00    12
Mo-Fr 08:00-19:00; Sa 08:00-13:00    10
Mo-Fr 08:30-19:00; Sa 09:00-14:00    10
Mo-Fr 09:00-18:00; Sa 09:00-13:00     9
Name: count, dtype: int64


In [158]:
# Example : most commmon building_type
print("\nTop building_type:")
print(pharmacy_df["building_type"].value_counts().head(10))


Top building_type:
building_type
yes           2
commercial    2
apartments    1
Name: count, dtype: int64


In [159]:
# Example: most commen wheelchair_accessible
print("\nTop wheelchair_accessible:")
print(pharmacy_df["wheelchair_accessible"].value_counts().head(10))


Top wheelchair_accessible:
wheelchair_accessible
yes        461
no          89
limited     68
Name: count, dtype: int64


In [160]:
# Example: Unique suriveillance types
print("\nUnique suriveillance:")
print(pharmacy_df["surveillance"].unique())


Unique suriveillance:
[nan 'outdoor' 'indoor' 'yes']


In [161]:
# Example: Unique delivery types
print("\nUnique delivery:")
print(pharmacy_df["delivery"].unique())


Unique delivery:
[nan 'yes']


In [162]:
# Example: Unique payment methods
print("\nUnique payment_visa:")
print(pharmacy_df["payment_visa"].unique())     
# Example: Unique payment_mastercard methods
print("\nUnique payment_mastercard:")
print(pharmacy_df["payment_mastercard"].unique())     
# Example: Unique payment_girocard methods
print("\nUnique payment_girocard:")
print(pharmacy_df["payment_girocard"].unique()) 


Unique payment_visa:
[nan 'yes']

Unique payment_mastercard:
[nan 'yes']

Unique payment_girocard:
[nan 'yes']


In [163]:
# check the unique values in dispensing
print("\nUnique dispensing:")
print(pharmacy_df["dispensing"].unique())
# check the unique values in building_type
print("\nUnique building_type:")
print(pharmacy_df["building_type"].unique())


Unique dispensing:
['yes' nan 'no' 'Apotheke in Nikolassee']

Unique building_type:
[nan 'yes' 'commercial' 'apartments']


### E) Geometry sanity checks

In [164]:
# Goal: Ensure spatial data makes sense.

# Unique geometry types (Point, Polygon/LineString). 
# If some are Polygon/LineString, I already handled them with .representative_point() (somewhere above in Step 1.2).

print(pharmacy_df.geometry.geom_type.value_counts())

Point    675
Name: count, dtype: int64


In [165]:
# Any missing geometries?
# Why? Missing geometry would be a problem for maps.

print("Missing geometries:", pharmacy_df.geometry.isna().sum())

Missing geometries: 0


### F) Latitude/Longitude checks

In [166]:
# Goal: Verify lat/lon look realistic.
# Why? If values are way off, something went wrong in conversion.

print("Latitude range:", pharmacy_df["latitude"].min(), "to", pharmacy_df["latitude"].max())

print("Longitude range:", pharmacy_df["longitude"].min(), "to", pharmacy_df["longitude"].max())


Latitude range: 52.3865161 to 52.6360319
Longitude range: 13.1422529 to 13.715511


## 1.3 Prepare the /sources Directory

- **Raw Data Files:**  
    - `pharmacies_raw.geojson` (includes geometry)  
    - `pharmacies_raw.csv` (tabular only, no geometry)  

- **README.md** in `/sources` will contain:
    - Data sources used.
    - Planned transformation steps.


In [167]:
import os

In [54]:
# Define file paths
raw_geojson_path = "../sources/pharmacies_raw.geojson"
raw_csv_path = "../sources/pharmacies_raw.csv"


In [55]:
# create folder to save dta
os.makedirs(os.path.dirname(raw_geojson_path), exist_ok=True)

In [ ]:
# Save as GeoJSON (keeps geometry) and CSV


pharmacy_gdf.to_file(raw_geojson_path, driver="GeoJSON")
pharmacy_gdf.drop(columns="geometry").to_csv(raw_csv_path, index=False)

print(f"Raw data saved to: {raw_geojson_path} and {raw_csv_path}")

Raw data saved to: ../sources/pharmacies_raw.geojson and ../sources/pharmacies_raw.csv


## 1.4 Review

- All 23 target columns defined.
- Data sources identified and documented.
- Schema draft created.
- Data fetched and stored in `/sources`.
- Data cleaning & enrichment plan in place.

**Next Step:** Step 2 — Fetch & Transform data.


# 🛠 Step 2: Data Transformation

In [169]:
#check the unique values in dispensing and delivery and wheelchair_accessible
print("\nUnique dispensing:")
print(pharmacy_df["dispensing"].unique())
print("\nUnique delivery:")
print(pharmacy_df["delivery"].unique())     
print("\nUnique wheelchair_accessible:")
print(pharmacy_df["wheelchair_accessible"].unique())




Unique dispensing:
['yes' nan 'no' 'Apotheke in Nikolassee']

Unique delivery:
[nan True]

Unique wheelchair_accessible:
['yes' 'no' 'limited' nan]


### A) Standardize column names and types

In [170]:
# Standardize column names

pharmacy_df.columns = pharmacy_df.columns.str.lower().str.strip().str.replace(" ", "_").str.replace("-", "_")

# Convert certain columns to correct type

pharmacy_df["housenumber"] = pharmacy_df["housenumber"].astype(str)   # ensure text

pharmacy_df["postcode"] = pharmacy_df["postcode"].astype(str)         # keep leading zeros

# Normalize yes/no columns into Boolean (True/False)

pharmacy_df["payment_visa"] = pharmacy_df["payment_visa"].map({"yes": True, "no": False})
pharmacy_df["payment_mastercard"] = pharmacy_df["payment_mastercard"].map({"yes": True, "no": False})
pharmacy_df["payment_girocard"] = pharmacy_df["payment_girocard"].map({"yes": True, "no": False})


pharmacy_df["surveillance"] = pharmacy_df["surveillance"].map({"yes": True, "no": False})
pharmacy_df["delivery"] = pharmacy_df["delivery"].map({"yes": True, "no": False})


# Make text values consistent (lowercase to avoid duplicates like "Sparkasse" vs "sparkasse")
# See "opening_hours" normalization in Step 2 E)


text_cols = ["name", "street", "city", "country", "website", "operator", "brand", "phone", "email", "source", "building"]
for col in text_cols:
    if col in pharmacy_df.columns:
        pharmacy_df[col] = pharmacy_df[col].astype(str).str.strip().str.lower()

Unique dispensing:
['yes' nan 'no' 'Apotheke in Nikolassee']

Unique delivery:
[nan 'yes']

Unique wheelchair_accessible:
['yes' 'no' 'limited' nan]

In [171]:
# Check the  datatypes after Step 2 A)

print(pharmacy_df.dtypes)   

name                       object
brand                      object
operator                   object
street                     object
housenumber                object
postcode                   object
suburb                     object
city                       object
country                    object
phone                      object
email                      object
website                    object
openinghours               object
payment_visa               object
payment_mastercard         object
payment_girocard           object
dispensing                 object
delivery                   object
surveillance               object
wheelchair_accessible      object
building_type              object
latitude                  float64
longitude                 float64
geometry                 geometry
source                     object
dtype: object


In [116]:
# See first rows after Step 2 A)

pharmacy_df.head() 


name brand operator  \
element id                                                
node    60775323  reichenberger apotheke   nan      nan   
        60848447       castello-apotheke   nan      nan   
        60852928          rosen apotheke   nan      nan   
        68437791     margareten-apotheke   nan      nan   
        69226035      leipziger apotheke   nan      nan   

                                  street housenumber postcode  \
element id                                                      
node    60775323    reichenberger straße           3    13055   
        60848447                     nan         nan      nan   
        60852928  rudolf-seiffert-straße          11    10369   
        68437791      karl-kunger-straße          46    12435   
        69226035        leipziger straße          43    10117   

                                suburb    city country           phone email  \
element id                                                                     
node    60775323  Alt-Hohenschönhausen  berlin      de  +49 30 9713807   nan   
        60848447                   NaN     nan     nan             nan   nan   
        60852928             Fennpfuhl  berlin      de  +49 30 9759449   nan   
        68437791           Alt-Treptow  berlin     nan  +49 30 5337855   nan   
        69226035                 Mitte  berlin      de             nan   nan   

                                                  website  \
element id                                                  
node    60775323        https://reichenbergerapotheke.de/   
        60848447                                      nan   
        60852928                  https://www.zurrose.de/   
        68437791  http://www.apotheke.borchert-online.de/   
        69226035       https://www.leipziger-apotheke.de/   

                                                       openinghours  \
element id                                                            
node    60775323  Mo-Th 08:00-19:00; Fr 08:00-18:30; Sa 09:00-13:00   
        60848447                  Mo-Fr 08:30-19:00; Sa 08:30-14:00   
        60852928                  Mo-Fr 08:00-19:00; Sa 08:00-12:00   
        68437791                  Mo-Fr 08:30-18:30; Sa 08:30-13:00   
        69226035                  Mo-Fr 08:00-19:00; Sa 08:00-14:00   

                 payment_visa payment_mastercard payment_girocard dispensing  \
element id                                                                     
node    60775323          NaN                NaN              NaN        yes   
        60848447          NaN                NaN              NaN        yes   
        60852928          NaN                NaN              NaN        yes   
        68437791          NaN                NaN              NaN        yes   
        69226035         True               True              NaN        yes   

                 delivery surveillance wheelchair_accessible building_type  \
element id                                                                   
node    60775323      NaN          NaN                   yes           NaN   
        60848447      NaN          NaN                   yes           NaN   
        60852928      NaN          NaN                   yes           NaN   
        68437791      NaN          NaN                    no           NaN   
        69226035      NaN          NaN                   yes           NaN   

                   latitude  longitude                   geometry source  
element id                                                                
node    60775323  52.540741  13.486602   POINT (13.4866 52.54074)    nan  
        60848447  52.531835  13.469654  POINT (13.46965 52.53184)    nan  
        60852928  52.527555  13.468513  POINT (13.46851 52.52756)    nan  
        68437791  52.489390  13.450570  POINT (13.45057 52.48939)    nan  
        69226035  52.510556  13.395863  POINT (13.39586 52.51056)    nan

In [ ]:
# Seeing more than head()

# pharmacy_df.head(20)              # first 20 rows
# pharmacy_df.tail(10)              # last 10 rows
# pharmacy_df.sample(10, random_state=0)  # 10 random rows
# pharmacy_df[["brand","operator","atm","wheelchair"]].sample(15, random_state=1)
# pharmacy_df["brand"].value_counts(dropna=False).head(20)


In [172]:
# Seeing more than head()

pharmacy_df.sample(10, random_state=0)  # 10 random rows

name         brand           operator  \
element id                                                                     
node    4381928183  apotheke am bundesplatz           nan    michaela kröger   
        263699380         robinson apotheke           nan                nan   
        1038217551          falken-apotheke           nan                nan   
        454167250          bartels apotheke           nan                nan   
        1624585364           vital-apotheke           nan                nan   
        738996191          kurmark apotheke           nan                nan   
        833390882              easyapotheke  easyapotheke  mehmet geyik e.k.   
        87036261          nordland apotheke           nan  tina töllner e.k.   
        627415668   gesundbrunnen- apotheke           nan                nan   
        442705525          hermann apotheke           nan                nan   

                                      street housenumber postcode  \
element id                                                          
node    4381928183               bundesplatz           3    10715   
        263699380   lion-feuchtwanger-straße          22    12619   
        1038217551           siegener straße          59    13583   
        454167250                        nan         nan      nan   
        1624585364          skalitzer straße          15    10999   
        738996191           kurfürstenstraße         154    10785   
        833390882               schloßstraße           1    12163   
        87036261             invalidenstraße         114    10115   
        627415668                  badstraße          64    13357   
        442705525              hermannstraße         116    12051   

                                suburb    city country             phone  \
element id                                                                 
node    4381928183                 NaN  berlin     nan   +49 30 85405670   
        263699380          Hellersdorf  berlin      de               nan   
        1038217551  Falkenhagener Feld  berlin      de               nan   
        454167250                  NaN     nan     nan   +49 30 47301356   
        1624585364           Kreuzberg  berlin      de               nan   
        738996191           Schöneberg  berlin      de  +4930 26 555 477   
        833390882             Steglitz  berlin      de   +49 30 79016052   
        87036261                   NaN  berlin     nan               nan   
        627415668        Gesundbrunnen  berlin      de               nan   
        442705525             Neukölln  berlin      de     +493062981014   

                                              email  \
element id                                            
node    4381928183                              nan   
        263699380                               nan   
        1038217551                              nan   
        454167250                               nan   
        1624585364                              nan   
        738996191                               nan   
        833390882                               nan   
        87036261    info@nordlandapotheke-berlin.de   
        627415668                               nan   
        442705525                               nan   

                                                     website  \
element id                                                     
node    4381928183      https://www.apotheke-bundesplatz.de/   
        263699380                                        nan   
        1038217551                                       nan   
        454167250           https://www.bartels-apotheke.de/   
        1624585364                                       nan   
        738996191   http://www.apo-net.de/kurmark/index2.htm   
        833390882   https://forum-steglitz.easyapotheken.de/   
        87036261         https://nordlandapotheke-berlin.de/   
        627415668                  

In [173]:
# Seeing more than head()

pharmacy_df[["brand","operator","wheelchair_accessible","delivery"]].sample(15, random_state=2)

brand                        operator  \
element id                                                 
node    6006371223   nan                             nan   
        1906515104   nan                             nan   
        3353252595   nan                             nan   
        1617419904   nan                             nan   
        313933591    nan                             nan   
        287852225    nan                             nan   
        3556800140   nan                             nan   
        297512117    nan              norbert peter e.k.   
        983454566    nan                             nan   
        667211398    nan                             nan   
        592122156    nan                dr. jutta wenger   
        456421034    nan                             nan   
        5181703340   nan  dr. sabine maria knoll-schütze   
        1236581643   nan                             nan   
        330966646    nan                             nan   

                   wheelchair_accessible delivery  
element id                                         
node    6006371223                   yes      NaN  
        1906515104                   yes      NaN  
        3353252595                   yes      NaN  
        1617419904                   yes      NaN  
        313933591                    NaN      NaN  
        287852225                    yes      NaN  
        3556800140                    no      NaN  
        297512117                limited      NaN  
        983454566                     no      NaN  
        667211398                    yes      NaN  
        592122156                limited      NaN  
        456421034                    yes      NaN  
        5181703340                   yes      NaN  
        1236581643                   yes      NaN  
        330966646                    yes      NaN

### B) Drop irrelevant / redundant columns

In [174]:
# # Drop redundant columns
columns_to_drop_in_2B = ["city", "country", "source"]

# Keep only the ones that really exist in the dataframe
columns_to_drop_in_2B = [col for col in columns_to_drop_in_2B if col in pharmacy_df.columns]

print("Dropping in Step 2B:", columns_to_drop_in_2B)
pharmacy_df = pharmacy_df.drop(columns=columns_to_drop_in_2B)

print("\nRemaining columns after Step 2B:")
print(pharmacy_df.columns.tolist())

Dropping in Step 2B: ['city', 'country', 'source']

Remaining columns after Step 2B:
['name', 'brand', 'operator', 'street', 'housenumber', 'postcode', 'suburb', 'phone', 'email', 'website', 'openinghours', 'payment_visa', 'payment_mastercard', 'payment_girocard', 'dispensing', 'delivery', 'surveillance', 'wheelchair_accessible', 'building_type', 'latitude', 'longitude', 'geometry']


### C) Handle missing values

In [175]:
# Drop columns with too many missing values => See table with counts and % of missing values in Step 1 B)
# delivery > 90% missing;
# building_type > 90% missing
# payment_girocard > 90% missing
# payment_visa > 90% missing
# payment_mastercard > 90% missing
# surveillance > 90% missing
# brand > 90% missing
# email > 80% missing


columns_to_drop_in_2C = ["delivery", "building_type", "payment_girocard", "payment_visa", "payment_mastercard", "surveillance", "brand", "email"]

columns_to_drop_in_2C = [col for col in columns_to_drop_in_2C if col in pharmacy_df.columns]

print("Dropping in Step 2C:", columns_to_drop_in_2C)
pharmacy_df = pharmacy_df.drop(columns=columns_to_drop_in_2C)

print("\nRemaining columns after Step 2C:")
print(pharmacy_df.columns.tolist())

Dropping in Step 2C: ['delivery', 'building_type', 'payment_girocard', 'payment_visa', 'payment_mastercard', 'surveillance', 'brand', 'email']

Remaining columns after Step 2C:
['name', 'operator', 'street', 'housenumber', 'postcode', 'suburb', 'phone', 'website', 'openinghours', 'dispensing', 'wheelchair_accessible', 'latitude', 'longitude', 'geometry']


In [194]:
# check the pharmacy_df after Step 2 C)
pharmacy_df.head()

,pharmacy_id,name,operator,street,housenumber,postcode,suburb,phone,website,openinghours,dispensing,wheelchair_accessible,latitude,longitude,district,district_id
0,60775323,reichenberger apotheke,nan,reichenberger straße,3,13055,Alt-Hohenschönhausen,+49 30 9713807,https://reichenbergerapotheke.de/,mo-th 08:00-19:00; fr 08:00-18:30; sa 09:00-13:00,yes,yes,52.540741,13.486602,Lichtenberg,11011011
1,60848447,castello-apotheke,nan,nan,nan,nan,NaN,nan,nan,mo-fr 08:30-19:00; sa 08:30-14:00,yes,yes,52.531835,13.469654,Lichtenberg,11011011
2,60852928,rosen apotheke,nan,rudolf-seiffert-straße,11,10369,Fennpfuhl,+49 30 9759449,https://www.zurrose.de/,mo-fr 08:00-19:00; sa 08:00-12:00,yes,yes,52.527555,13.468513,Lichtenberg,11011011
3,68437791,margareten-apotheke,nan,karl-kunger-straße,46,12435,Alt-Treptow,+49 30 5337855,http://www.apotheke.borchert-online.de/,mo-fr 08:30-18:30; sa 08:30-13:00,yes,no,52.489390,13.450570,Treptow-Köpenick,11009009
4,69226035,leipziger apotheke,nan,leipziger straße,43,10117,Mitte,nan,https://www.leipziger-apotheke.de/,mo-fr 08:00-19:00; sa 08:00-14:00,yes,yes,52.510556,13.395863,Mitte,11001001


In [195]:
# check missing values after Step 2 C)
missing_count = pharmacy_df.isna().sum().sort_values(ascending=False)
print(missing_count[missing_count > 0])

suburb                   231
dispensing                90
wheelchair_accessible     57
dtype: int64


### D) Normalize categories

### E) Opening hours normalization

In [178]:
#check the unique values in openinghours
print("\nUnique openinghours:")
print(pharmacy_df["openinghours"].unique()) 


Unique openinghours:
['Mo-Th 08:00-19:00; Fr 08:00-18:30; Sa 09:00-13:00'
 'Mo-Fr 08:30-19:00; Sa 08:30-14:00' 'Mo-Fr 08:00-19:00; Sa 08:00-12:00'
 'Mo-Fr 08:30-18:30; Sa 08:30-13:00' 'Mo-Fr 08:00-19:00; Sa 08:00-14:00'
 'Mo-Fr 08:00-18:30; Sa 08:00-13:00' 'Mo-Fr 08:30-18:30; Sa 09:00-13:00'
 'Mo-Fr 09:00-19:00; Sa 09:00-14:00' 'Mo-Fr 08:30-19:00; Sa 09:00-13:00'
 'Mo-Fr 09:00-19:00; Sa 10:00-19:00' 'Mo-Fr 08:30-18:00, Sa 08:30-13:00'
 'Mo-Fr 08:30-18:30, Sa 08:30-13:00' 'Mo-Fr 08:30-18:30'
 'Mo-Fr 08:00-19:00' 'Mo-Fr 08:00-19:00; Sa 09:00-13:00'
 'Mo-Fr 08:00-20:00; Sa 08:30-18:00' 'Mo-Fr 09:00-19:30; Sa 10:00-19:00'
 'Mo-Fr 08:00-19:00; Sa 08:30-14:00' nan
 'Mo-Fr 08:30-19:00;Sa 08:30-13:00;Su,PH off'
 'Mo-Fr 09:00-18:30; Sa 09:00-13:00'
 'Mo,Tu 08:30-19:00; We 08:30-18:30; Th 08:30-19:00; Fr 08:30-18:30'
 'Mo-Fr 08:00-18:30; Sa 09:00-14:00' 'Mo-Fr 09:00-18:30, Sa 10:30-16:30'
 'Mo-Fr 09:00-18:00; Sa 09:00-14:00'
 'Mo-Fr 08:00-20:00; Sa 09:00-15:00; PH off'
 'Mo-Fr 08:00-19:00, Sa 0

In [181]:
# Normalize text format

if "openinghours" in pharmacy_df.columns:
    pharmacy_df["openinghours"] = pharmacy_df["openinghours"].astype(str).str.strip().str.lower()

print("\nSample opening hours values:")
print(pharmacy_df["openinghours"].head(10) if "openinghours" in pharmacy_df.columns else "No column")


Sample opening hours values:
element  id      
node     60775323    mo-th 08:00-19:00; fr 08:00-18:30; sa 09:00-13:00
         60848447                    mo-fr 08:30-19:00; sa 08:30-14:00
         60852928                    mo-fr 08:00-19:00; sa 08:00-12:00
         68437791                    mo-fr 08:30-18:30; sa 08:30-13:00
         69226035                    mo-fr 08:00-19:00; sa 08:00-14:00
         76507297                    mo-fr 08:00-18:30; sa 08:00-13:00
         76519952                    mo-fr 08:30-18:30; sa 09:00-13:00
         76596388                    mo-fr 09:00-19:00; sa 09:00-14:00
         78244254                    mo-fr 08:30-19:00; sa 09:00-13:00
         79603436                    mo-fr 09:00-19:00; sa 10:00-19:00
Name: openinghours, dtype: object


In [182]:
# Quick preview

pharmacy_df.head()


name operator                  street  \
element id                                                                  
node    60775323  reichenberger apotheke      nan    reichenberger straße   
        60848447       castello-apotheke      nan                     nan   
        60852928          rosen apotheke      nan  rudolf-seiffert-straße   
        68437791     margareten-apotheke      nan      karl-kunger-straße   
        69226035      leipziger apotheke      nan        leipziger straße   

                 housenumber postcode                suburb           phone  \
element id                                                                    
node    60775323           3    13055  Alt-Hohenschönhausen  +49 30 9713807   
        60848447         nan      nan                   NaN             nan   
        60852928          11    10369             Fennpfuhl  +49 30 9759449   
        68437791          46    12435           Alt-Treptow  +49 30 5337855   
        69226035          43    10117                 Mitte             nan   

                                                  website  \
element id                                                  
node    60775323        https://reichenbergerapotheke.de/   
        60848447                                      nan   
        60852928                  https://www.zurrose.de/   
        68437791  http://www.apotheke.borchert-online.de/   
        69226035       https://www.leipziger-apotheke.de/   

                                                       openinghours  \
element id                                                            
node    60775323  mo-th 08:00-19:00; fr 08:00-18:30; sa 09:00-13:00   
        60848447                  mo-fr 08:30-19:00; sa 08:30-14:00   
        60852928                  mo-fr 08:00-19:00; sa 08:00-12:00   
        68437791                  mo-fr 08:30-18:30; sa 08:30-13:00   
        69226035                  mo-fr 08:00-19:00; sa 08:00-14:00   

                 dispensing wheelchair_accessible   latitude  longitude  \
element id                                                                
node    60775323        yes                   yes  52.540741  13.486602   
        60848447        yes                   yes  52.531835  13.469654   
        60852928        yes                   yes  52.527555  13.468513   
        68437791        yes                    no  52.489390  13.450570   
        69226035        yes                   yes  52.510556  13.395863   

                                   geometry  
element id                                   
node    60775323   POINT (13.4866 52.54074)  
        60848447  POINT (13.46965 52.53184)  
        60852928  POINT (13.46851 52.52756)  
        68437791  POINT (13.45057 52.48939)  
        69226035  POINT (13.39586 52.51056)

In [184]:
#check shape
pharmacy_df.shape

(675, 14)

### F) Add district and district_id to the data frame

In [183]:
conda install -c conda-forge geopy


2 channel Terms of Service accepted
Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/p310

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-2.1          |     pyhd8ed1ab_0          40 KB  conda-forge
    geopy-2.4.1                |     pyhd8ed1ab_2          71 KB  conda-forge
    openssl-3.5.3              |       h5503f6c_1         2.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  g

In [186]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

# Initialize the geolocator
geolocator = Nominatim(user_agent="berlin_district_locator")

# Define a reverse geocoding function to get the official Berlin district (Bezirk)
def get_district(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language='de')
        sleep(1)  # Nominatim rate limit: 1 request per second
        if location and "address" in location.raw:
            address = location.raw["address"]
            return (
                address.get("city_district") or
                address.get("borough") or
                address.get("county") or
                None
            )
        return None
    except:
        return None

# Apply function row by row → add new "district" column
pharmacy_df["district"] = pharmacy_df.apply(
    lambda row: get_district(row["latitude"], row["longitude"]) if pd.notnull(row["latitude"]) else None,
    axis=1
)




In [ ]:
# pharmacy_df = pharmacy_df.drop(columns="district_id")

In [187]:
# Generating district ids
# https://www.regionalstatistik.de

# District mapping (official codes as strings)
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column (string)
pharmacy_df['district_id'] = pharmacy_df['district'].map(district_mapping).astype(str)

# (Optional) Check if some districts were not mapped
#unmapped = df[~df['district'].isin(district_mapping.keys())]['district'].unique()
#if len(unmapped) > 0:
    #print("⚠️ Unmapped districts found:", unmapped)

### G)  Reset index, drop columns "element" and "geometry", rename "id" to "banks_id"

In [188]:
# Reset index
pharmacy_df= pharmacy_df.drop(columns=["geometry"]).reset_index()

In [189]:
# Drop the redundant column "element" 

pharmacy_df= pharmacy_df.drop(columns=["element"])

In [190]:
# Rename the "id" column to "bank_id"

pharmacy_df = pharmacy_df.rename(columns={"id": "pharmacy_id"})

In [191]:
# Change bank_id column type to string

pharmacy_df["pharmacy_id"] = pharmacy_df["pharmacy_id"].astype(str)



In [192]:
# (Optional) Save enriched dataset for later use
pharmacy_df.to_csv("pharmacy_with_districts.csv", index=False)

### H)  Final Summary of Cleaned and Transformed Data

In [193]:
print("✅ Dataset after Steps A - G cleaning and transforming\n")

# Shape of dataframe
print(f"Number of rows: {pharmacy_df.shape[0]}")
print(f"Number of columns: {pharmacy_df.shape[1]}")

# Column list
print("\nRemaining columns:")
print(pharmacy_df.columns.tolist())

# Missing values check
missing = pharmacy_df.isnull().sum()
print("\nMissing values after cleaning and transforming :")
print(missing)

✅ Dataset after Steps A - G cleaning and transforming

Number of rows: 675
Number of columns: 16

Remaining columns:
['pharmacy_id', 'name', 'operator', 'street', 'housenumber', 'postcode', 'suburb', 'phone', 'website', 'openinghours', 'dispensing', 'wheelchair_accessible', 'latitude', 'longitude', 'district', 'district_id']

Missing values after cleaning and transforming :
pharmacy_id                0
name                       0
operator                   0
street                     0
housenumber                0
postcode                   0
suburb                   231
phone                      0
website                    0
openinghours               0
dispensing                90
wheelchair_accessible     57
latitude                   0
longitude                  0
district                   0
district_id                0
dtype: int64


In [197]:
pharmacy_df.head()

,pharmacy_id,name,operator,street,housenumber,postcode,suburb,phone,website,openinghours,dispensing,wheelchair_accessible,latitude,longitude,district,district_id
0,60775323,reichenberger apotheke,nan,reichenberger straße,3,13055,Alt-Hohenschönhausen,+49 30 9713807,https://reichenbergerapotheke.de/,mo-th 08:00-19:00; fr 08:00-18:30; sa 09:00-13:00,yes,yes,52.540741,13.486602,Lichtenberg,11011011
1,60848447,castello-apotheke,nan,nan,nan,nan,NaN,nan,nan,mo-fr 08:30-19:00; sa 08:30-14:00,yes,yes,52.531835,13.469654,Lichtenberg,11011011
2,60852928,rosen apotheke,nan,rudolf-seiffert-straße,11,10369,Fennpfuhl,+49 30 9759449,https://www.zurrose.de/,mo-fr 08:00-19:00; sa 08:00-12:00,yes,yes,52.527555,13.468513,Lichtenberg,11011011
3,68437791,margareten-apotheke,nan,karl-kunger-straße,46,12435,Alt-Treptow,+49 30 5337855,http://www.apotheke.borchert-online.de/,mo-fr 08:30-18:30; sa 08:30-13:00,yes,no,52.489390,13.450570,Treptow-Köpenick,11009009
4,69226035,leipziger apotheke,nan,leipziger straße,43,10117,Mitte,nan,https://www.leipziger-apotheke.de/,mo-fr 08:00-19:00; sa 08:00-14:00,yes,yes,52.510556,13.395863,Mitte,11001001


In [198]:
# Data types and non-null counts

pharmacy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pharmacy_id            675 non-null    object 
 1   name                   675 non-null    object 
 2   operator               675 non-null    object 
 3   street                 675 non-null    object 
 4   housenumber            675 non-null    object 
 5   postcode               675 non-null    object 
 6   suburb                 444 non-null    object 
 7   phone                  675 non-null    object 
 8   website                675 non-null    object 
 9   openinghours           675 non-null    object 
 10  dispensing             585 non-null    object 
 11  wheelchair_accessible  618 non-null    object 
 12  latitude               675 non-null    float64
 13  longitude              675 non-null    float64
 14  district               675 non-null    object 
 15  distri

In [199]:
pharmacy_df.to_csv("final_pharmacies_with_districts.csv")

# Step 3 does not perform until approved

# 🧩 Step 3: Populate Database

In [200]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [201]:
user_name='clara_neagu'
password='nG60A6GG8ZKZuV'

In [202]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

In [203]:
#this is where you create table with constraints and references first
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.banks (
    pharmacy_id VARCHAR(20) PRIMARY KEY,
    name VARCHAR(255),
    operator VARCHAR(255),
    street VARCHAR(255),
    housenumber VARCHAR(50),
    postcode VARCHAR(10),
    suburb VARCHAR(100),
    phone VARCHAR(100),
    website VARCHAR(255),
    openinghours TEXT,
    dispensing VARCHAR(10),
    wheelchair VARCHAR(10),
    latitude FLOAT,
    longitude FLOAT,
    district VARCHAR(100),
    district_id VARCHAR(100),
     CONSTRAINT district_id_fk FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
   
);
"""

# Execute the query to create empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction




OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5433 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
#  Send the DataFrame to the database using .to_sql()
pharmacy_df.to_sql(
    'pharmacies',      
    engine,
    schema=schema,
    if_exists='append', # ✅ keeps table, just inserts data
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!


In [137]:
##let's query test data!
query = f"""
SELECT * from berlin_source_data.banks
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,bank_id,name,brand,operator,street,housenumber,postcode,opening_hours,atm,wheelchair,latitude,longitude,district,district_id
0,28968292,berliner volksbank,berliner volksbank,nan,berliner straße,42,10713,"mo-fr 10:00-13:00, mo 14:00-16:00, tu,th 14:00...",true,true,52.486668,13.319723,Charlottenburg-Wilmersdorf,11004004
1,60848455,sparkasse,nan,berliner sparkasse,anton-saefkow-platz,13,10369,"mo,we,fr 09:30-15:00; tu,th 09:30-18:00",true,unknown,52.530331,13.471037,Lichtenberg,11011011
2,87040399,dkb,nan,nan,nan,nan,nan,unknown,unknown,unknown,52.511050,13.388798,Mitte,11001001
3,89274635,deutsche bank,deutsche bank,deutsche bank,alexanderstraße,5,10178,mo-tu 10:00-18:00; we 10:00-16:00; th 10:00-18...,true,true,52.523238,13.415750,Mitte,11001001
4,203561614,sparkasse,nan,berliner sparkasse,helene-weigel-platz,1/2,12681,"mo,we,fr 09:30-15:00; tu,th 09:30-18:00",true,true,52.527687,13.538327,Marzahn-Hellersdorf,11010010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,210920895,sparkasse,nan,berliner sparkasse,alt-rudow,15,12357,"mo,we,fr 09:30-15:00; tu,th 09:30-18:00",true,true,52.420953,13.491388,Neukölln,11008008
319,336063499,reisebank,reisebank,reisebank,am ostbahnhof,5,10243,mo-fr 09:00-19:00; sa 10:00-18:00,true,true,52.510354,13.433634,Friedrichshain-Kreuzberg,11002002
320,422475544,allgemeine beamten bank,nan,nan,invalidenstraße,28,10115,"mo-fr 09:00-13:00, mo,we 14:00-18:00",unknown,true,52.531277,13.384913,Mitte,11001001
321,423847739,sparda-bank berlin,sparda-bank berlin,sparda-bank berlin eg,nan,nan,nan,"mo,th 09:00-13:00,14:00-18:00",true,true,52.534183,13.198554,Spandau,11005005
